#  YaProfi baseline

In [1]:
USER = !pwd
USER = USER[0]
USER

'/home/rvmusaev'

In [2]:
%cd $USER

/home/rvmusaev


In [3]:
!mkdir /home/repo
%cd /home/repo
!git clone https://github.com/naver/r2d2.git
!git clone https://github.com/naver/deep-image-retrieval.git
!git clone https://github.com/naver/kapture.git kapture
!git clone https://github.com/naver/kapture-localization.git
%cd $USER

mkdir: cannot create directory ‘/home/repo’: File exists
/home/repo
fatal: destination path 'r2d2' already exists and is not an empty directory.
fatal: destination path 'deep-image-retrieval' already exists and is not an empty directory.
fatal: destination path 'kapture' already exists and is not an empty directory.
fatal: destination path 'kapture-localization' already exists and is not an empty directory.
/home/rvmusaev


In [4]:
!unzip $USER/model/Resnet101-AP-GeM-LM18.pt.zip
!mv Resnet101-AP-GeM-LM18.pt /home/repo/deep-image-retrieval

Archive:  /home/rvmusaev/model/Resnet101-AP-GeM-LM18.pt.zip
replace Resnet101-AP-GeM-LM18.pt? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C
mv: cannot create regular file '/home/repo/deep-image-retrieval/Resnet101-AP-GeM-LM18.pt': Permission denied


In [10]:
import os
from tqdm import tqdm

class Structure():
    
    '''
    Make appropriate for Kapture structure :
    
    kapture_dir/
        query/
            sensors/
                ...
        mapping/
            sensors/
                ...
    '''

    def __init__(self, root):
   
        self.root = root
        self.frequency = {'query': 1, 'mapping': 20}
        self.dates = {'mapping':['2021-03-27-09-08-15','2021-03-27-10-17-01', '2021-03-27-10-32-53'],'query':['2021-03-27-03-30-47']}

    def mkstructure(self):
        
        !mkdir $self.root/query/sensors
        !mkdir $self.root/mapping/sensors
        !mkdir $self.root/query/sensors/records_data
        !mkdir $self.root/mapping/sensors/records_data
        modes = ['query','mapping']
        for mode in modes:
            freq = self.frequency[mode]
            records_camera = open(f'{self.root}/{mode}/sensors/records_camera.txt', "w")
            records_camera.write('# timestamp, device_id, image_path\n')
            
            sensor =  open(f'{self.root}/{mode}/sensors/sensors.txt', 'w')
            sensor.write('# sensor_id, name, sensor_type, [sensor_params]+\n')
            
            trajectories = open(f'{self.root}/{mode}/sensors/trajectories.txt','w')
            trajectories.write('# timestamp, device_id, tx, ty, tz, qw, qx, qy, qz\n')
            
            
            dates = sorted(self.dates[mode])
            count = 0  ###################### !!!!! true timestaps will be better
            for date in dates:
                images = sorted(os.listdir(f'{self.root}/{mode}/{date}/rgb_left'))
                !cp $self.root/$mode/$date/rgb_left/* $self.root/$mode/sensors/records_data
                for index,image in tqdm(enumerate(images)):
                    if index % freq == 0:
                        string_records_camera = f'{count}, sensor{count}, {self.root}/{mode}/sensors/records_data/rgb_left/{image}' +'\n'
                        records_camera.write(string_records_camera)

                        string_sensor = f'sensor{count}, , camera, PINHOLE, 1280, 720, 684.06805, 684.06805, 615.11951 , 345.32159 \n'
                        sensor.write(string_sensor)
                        count+=1 
                    
 

            records_camera.close()
            sensor.close()
        
        
        gt_train =  sorted(list(map(lambda x: 'f{USER}/YaProfi/mapping/{x}/gt_tum.txt' ,self.dates['mapping'])))
        for file_path in self.gt_train:
            with open(file_path,'r') as f:
                for string in f.readlines():
                    timestamp, tx, ty, tz, qw, qx, qy, qz = string.split(' ')
                    qz = qz.split('\n')[0]
                    trajectories.write(f'{timestamp}, sensor, {tx}, {ty}, {tz}, {qw}, {qx}, {qy}, {qz}' +'\n')

        trajectories.close()


In [11]:
path = f'{USER}/YaProfi'
structure = Structure(path)
structure.mkstructure()

mkdir: cannot create directory ‘/home/rvmusaev/YaProfi/query/sensors’: File exists
mkdir: cannot create directory ‘/home/rvmusaev/YaProfi/mapping/sensors’: File exists
mkdir: cannot create directory ‘/home/rvmusaev/YaProfi/query/sensors/records_data’: File exists
mkdir: cannot create directory ‘/home/rvmusaev/YaProfi/mapping/sensors/records_data’: File exists


PermissionError: [Errno 13] Permission denied: '/home/rvmusaev/YaProfi/query/sensors/records_camera.txt'

In [17]:
class FeatureExctractor():
  
  def __init__(self, root):
    self.root = root
    self.modes = ['query', 'mapping']

  def extract_global_features(self):
    for mode in self.modes:
        kapture_root = USER + f'/YaProfi/{mode}'
        print(kapture_root)
        %cd $self.root/deep-image-retrieval
        !python3 -m dirtorch.extract_kapture --kapture-root $kapture_root --checkpoint Resnet101-AP-GeM-LM18.pt --gpu 0
        %cd f'/home/{USER}/YaProfi'

  def extract_local_features(self, TOP_K = 5000, min_size = 225, min_scale = 0.3, max_scale = 1.0):
    for mode in self.modes:
        kapture_root = USER + f'/YaProfi/{mode}'
        !python3 $self.root/r2d2/extract_kapture.py --kapture-root $kapture_root --model $self.root/r2d2/models/r2d2_WASF_N8_big.pt 




In [14]:
path = '/home/repo'
extractor = FeatureExctractor(path)
extractor.extract_global_features()
extractor.extract_local_features()

/home/rvmusaev/YaProfi/query
/home/repo/deep-image-retrieval
Launching on GPUs 0
=> loading checkpoint 'Resnet101-AP-GeM-LM18.pt' (current_iter 376)
loading /home/rvmusaev/YaProfi/query
All global features are already extracted
/home/repo
/home/rvmusaev/YaProfi/mapping
/home/repo/deep-image-retrieval
Launching on GPUs 0
=> loading checkpoint 'Resnet101-AP-GeM-LM18.pt' (current_iter 376)
loading /home/rvmusaev/YaProfi/mapping
Traceback (most recent call last):
  File "/usr/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/repo/deep-image-retrieval/dirtorch/extract_kapture.py", line 120, in <module>
    threads=args.threads, whiten=args.whiten)
  File "/home/repo/deep-image-retrieval/dirtorch/extract_kapture.py", line 29, in extract_kapture_global_features
    kapture.Observations])
  File "/usr/local/lib/python3.6/dist-packages/kapture/io/csv.py", line 

In [60]:
from kapture.io.features import keypoints_to_filepaths, image_keypoints_from_file
from kapture.io.records import images_to_filepaths
from kapture.io.image import image_keypoints_to_image_file
import kapture.io.csv as csv

kapture_data = csv.kapture_from_dir(f'{USER}/YaProfi/query')
kapture_data

sensors        : 13424
records_camera : 13424

## RESULTS

In [15]:
mapping = f'{USER}/YaProfi/mapping' 
query = f'{USER}/YaProfi/query' 
!python3 /home/repo/kapture-localization/tools/kapture_compute_image_pairs.py --query $query --mapping $mapping -o pairs_mapping_3.txt --topk 3 

Traceback (most recent call last):
  File "/home/repo/kapture-localization/tools/kapture_compute_image_pairs.py", line 137, in <module>
    compute_image_pairs_command_line()
  File "/home/repo/kapture-localization/tools/kapture_compute_image_pairs.py", line 133, in compute_image_pairs_command_line
    compute_image_pairs(args.mapping, args.query, args.output, args.topk)
  File "/home/repo/kapture-localization/tools/kapture_compute_image_pairs.py", line 44, in compute_image_pairs
    kapture.Points3d])
  File "/home/repo/kapture/kapture/io/csv.py", line 1294, in kapture_from_dir
    _load_all_records(csv_file_paths, kapture_loadable_data, kapture_data)
  File "/home/repo/kapture/kapture/io/csv.py", line 1332, in _load_all_records
    kapture_data.records_camera = records_camera_from_file(csv_file_paths[kapture.RecordsCamera], sensor_ids)
  File "/home/repo/kapture/kapture/io/csv.py", line 457, in records_camera_from_file
    records_camera[(int(timestamp), str(device_id))] = image_path

In [ ]:
imageslist = []
traject_list = []

with open('/home/repo/kapture-localization/samples/YaProfi/mapping/sensors/records_camera.txt') as fin:
  for j in fin.readlines():
    imageslist.append(j.split(', ')[-1].split('\n')[0])


with open(f'{USER}/YaProfi/mapping/trajectories.txt') as fin:
  for j in fin.readlines()[1:]:
    traject_list.append(tuple(j.split(', ')[2:]))


image_to_trajectories = {}
for key, value in zip(imageslist, traject_list):
  image_to_trajectories[key] = value

image_to_trajectories

In [ ]:
list_matches_Ap_Gem = []
answer_loc = []
with open('pairs_mapping_3.txt','r') as f:
  for line in f:
    list_matches_Ap_Gem.append(line)
    
list_matches_Ap_Gem = list_matches_Ap_Gem[2:]

#import re
#list_matches_Ap_Gem = sorted(list_matches_Ap_Gem,key=lambda x: (re.split(r'left',x.split(', ')[0])[0],int(re.split(r'left', x.split(', ')[0])[2].split('.')[0])))
  
s = slice(0, len(sorted(list_matches_Ap_Gem,key = lambda x: x[0])),3)
indices = range(s.start, s.stop, s.step)
it = iter(indices)


for pair in it:
  ap_gem = sorted(list_matches_Ap_Gem,key = lambda x: x[0])[pair:pair+3]
  ap_gem_query, ap_gem_mapping1, ap_gem_metric1 = ap_gem[0].split(', ')
  answer_loc.append(image_to_trajectories[ap_gem_mapping1])

In [ ]:
answer = list(map(lambda x:x[:3]), answer_loc)
submission = open('submission.txt','w')
for frame in answer:
    string = ' 'os.path.join(*frame)
    submission.write(string)
    
submission.close()